In [14]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import html

In [4]:
df = pd.read_csv('../data_collecting/raw_data.csv')
df

,tweetId,author_id,tweet,lang,created_at
0,1490905225930190849,2578185314,"In 2021, Reliance acquired @faradion_uk Ltd, l...",en,2022-02-08 04:28:00+00:00
1,1490905219286245376,291609629,"Hong Kong to tighten social-distancing rules, ...",en,2022-02-08 04:27:58+00:00
2,1490905197375553537,1320755164827938816,"you would know if someone lie, why those peopl...",en,2022-02-08 04:27:53+00:00
3,1490905197123620864,983016607,Doing. Experience and environment may shape yo...,en,2022-02-08 04:27:53+00:00
4,1490905176953454597,2675645600,@jenhada @airallianceHOU How does the communit...,en,2022-02-08 04:27:48+00:00
...,...,...,...,...,...
9995,1490810734426480642,56917394,🤔🙄 He sent an apology email 🤷🏼‍♀️. Biden's top...,en,2022-02-07 22:12:31+00:00
9996,1490810731167510530,66621747,"""80% of Hawaii residents say the Red Hill fuel...",en,2022-02-07 22:12:30+00:00
9997,1490810691150049280,378510560,It’s great to open up these views to Hilbre Is...,en,2022-02-07 22:12:21+00:00
9998,1490810668035026948,877523992155897856,@jbouie This one might be a tough one to make ...,en,2022-02-07 22:12:15+00:00


In [5]:
# check is all the data in english
tweet_by_lang = df.groupby("lang")["tweet"].count()
print(tweet_by_lang)

lang
en    10000
Name: tweet, dtype: int64


In [22]:
import numpy as np
import re

# need to use a stopword library
stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]


def clean_tweet(tweet):
    temp = tweet.lower()
    temp = temp.replace('\n', ' ')
    temp = html.unescape(temp)
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # removing handle 
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # removing hastags
    temp = re.sub(r'http\S+', '', temp) # removing any link
    temp = re.sub('[()!?]', ' ', temp)  # remove the punct 
    temp = re.sub('\[.*?\]',' ', temp)  # remove any weird symbol
    temp = re.sub("[^a-z0-9]"," ", temp) # remove any numbers
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords] 
    temp = " ".join(word for word in temp)
    return temp

In [23]:
results = [clean_tweet(tw) for tw in df['tweet']]

In [17]:
df['clean_tweet'] = results

In [18]:
df.head()

,tweetId,author_id,tweet,lang,created_at,clean_tweet
0,1490905225930190849,2578185314,"In 2021, Reliance acquired @faradion_uk Ltd, l...",en,2022-02-08 04:28:00+00:00,2021 reliance acquired ltd leading global sodi...
1,1490905219286245376,291609629,"Hong Kong to tighten social-distancing rules, ...",en,2022-02-08 04:27:58+00:00,hong kong tighten social distancing rules stan...
2,1490905197375553537,1320755164827938816,"you would know if someone lie, why those peopl...",en,2022-02-08 04:27:53+00:00,you would know if someone lie why those people...
3,1490905197123620864,983016607,Doing. Experience and environment may shape yo...,en,2022-02-08 04:27:53+00:00,doing experience environment may shape you nam...
4,1490905176953454597,2675645600,@jenhada @airallianceHOU How does the communit...,en,2022-02-08 04:27:48+00:00,how does community hold valero accountable tox...


In [19]:
df.to_csv('clean_data.csv', index=False)